In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import missingno as msn
import datetime
from pandas_profiling import ProfileReport
import sklearn
from sklearn.impute import SimpleImputer

In [65]:
df = pd.read_parquet("escooter_history_2022.parquet")



In [66]:
df.loc[df.temp > 50, "temp"] = df.temp.div(100)

In [67]:
df = pd.get_dummies(df)

In [44]:
df = df[df["registered_customer"] == False].copy()

In [68]:
df_2 = df.groupby(["registered_customer",pd.Grouper(key="datetime",freq="D")]).agg(Anzahl_pro_Tag=("registered_customer","count"),temp=("temp","mean"),humidity=("humidity","mean"),windspeed=("windspeed","mean"),workingday=("workingday","mean"),holiday=("holiday","mean"),Anzahl_registered=("registered_customer","sum"),clear_few_clouds=("weather_clear, few clouds","mean"),cloudy_mist=("weather_cloudy, mist","mean"),heacy_rain_thunderstorm_snow_icepallets=("weather_heacy rain or thunderstorm or snow or ice pallets","mean"),ligth_snow_rain_thunderstorm=("weather_light snow or rain or thunderstorm","mean")).reset_index()

In [69]:
df_2

,registered_customer,datetime,Anzahl_pro_Tag,temp,humidity,windspeed,workingday,holiday,Anzahl_registered,clear_few_clouds,cloudy_mist,heacy_rain_thunderstorm_snow_icepallets,ligth_snow_rain_thunderstorm
0,False,2020-01-04,331,16.868218,79.021148,17.235598,0.0,0.0,0,0.296073,0.658610,0.0,0.045317
1,False,2020-01-05,131,14.390687,67.244275,16.198554,0.0,0.0,0,0.183206,0.572519,0.0,0.244275
2,False,2020-01-06,120,9.129333,36.758333,15.966826,1.0,0.0,0,1.000000,0.000000,0.0,0.000000
3,False,2020-01-07,108,10.120926,53.638889,14.749616,1.0,0.0,0,0.953704,0.046296,0.0,0.000000
4,False,2020-01-08,82,10.390000,35.390244,15.500532,1.0,0.0,0,1.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,True,2021-12-30,4024,10.463559,59.591700,22.123152,1.0,0.0,4024,0.543986,0.450298,0.0,0.005716
1458,True,2021-12-31,7629,10.397415,58.792896,10.275610,1.0,0.0,7629,0.328352,0.671648,0.0,0.000000
1459,True,2022-01-01,5293,10.553724,76.329681,8.286138,0.0,0.0,5293,0.213678,0.625165,0.0,0.161156
1460,True,2022-01-02,4392,10.754472,46.234517,24.047083,0.0,0.0,4392,0.765255,0.234745,0.0,0.000000


In [20]:
df_2.dtypes

datetime             datetime64[ns]
Anzahl_pro_Tag                int64
temp                        float64
humidity                    float64
windspeed                   float64
workingday                  float64
holiday                     float64
Anzahl_registered             Int64
Anteil_registered           Float64
weekend                        bool
date_delta                    int64
dtype: object

In [70]:
df_2["workingday"] = df_2["workingday"].round()

In [71]:

df_2["holiday"] = df_2["holiday"].round() 

In [72]:
df_2["Anteil_registered"] = df_2["Anzahl_registered"]/df_2["Anzahl_pro_Tag"]

In [73]:
df_2["weekend"] = df_2["datetime"].dt.weekday > 4

In [74]:
df_2['date_delta'] = (df_2['datetime'] - df_2['datetime'].min())  
df_2["date_delta"] = df_2.date_delta.dt.days

In [ ]:
df_2.drop("Anzahl_registered",axis=1)

In [34]:
df_2["log_temp"] = np.log(df_2.temp)

In [ ]:
pd.plotting.scatter_matrix(df_2['temp', 'atemp',
       'humidity', 'windspeed',"Anzahl_pro_Tag"])

In [75]:
#trainings-Test-Split
features = ["registered_customer","date_delta", 'temp', 'humidity','windspeed',"workingday","holiday","weekend","clear_few_clouds","cloudy_mist","heacy_rain_thunderstorm_snow_icepallets","ligth_snow_rain_thunderstorm"] # Liste von Features
X = df_2[features]
y = df_2.Anzahl_pro_Tag

In [36]:
X

,date_delta,temp,log_temp,humidity,windspeed,workingday,holiday,weekend,clear_few_clouds,cloudy_mist,heacy_rain_thunderstorm_snow_icepallets,ligth_snow_rain_thunderstorm
0,0,16.298440,2.791069,80.010703,15.715434,0.0,0.0,True,0.316514,0.596330,0.0,0.087156
1,1,14.419761,2.668600,67.120896,15.811900,0.0,0.0,True,0.210448,0.510448,0.0,0.279104
2,2,8.372807,2.124989,40.632221,14.603443,1.0,0.0,False,1.000000,0.000000,0.0,0.000000
3,3,8.972063,2.194116,56.616919,12.173591,1.0,0.0,False,0.974553,0.025447,0.0,0.000000
4,4,9.609341,2.262736,38.383399,14.038239,1.0,0.0,False,1.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
726,726,10.463559,2.347899,59.591700,22.123152,1.0,0.0,False,0.543986,0.450298,0.0,0.005716
727,727,10.397415,2.341557,58.792896,10.275610,1.0,0.0,False,0.328352,0.671648,0.0,0.000000
728,728,10.553724,2.356479,76.329681,8.286138,0.0,0.0,True,0.213678,0.625165,0.0,0.161156
729,729,10.754472,2.375322,46.234517,24.047083,0.0,0.0,True,0.765255,0.234745,0.0,0.000000


In [59]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=False) # with_mean=False: dont rescale the means to being all zero
Xs = pd.DataFrame(scaler.fit_transform(X[X.columns]))
Xs.columns = X.columns

In [32]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [60]:
regr = linear_model.LinearRegression()
regr.fit(Xs, y)
pred = regr.predict(Xs)
print(Xs.columns.values)
print('Coefficients: \n', regr.coef_)
print('Intercept: \n', regr.intercept_)
print('On Training set:')
print("Mean squared error: %.2f" % mean_squared_error(y, pred))
print("Variance of target variable: %.2f" % np.var(y))
print('Variance explained: %.2f' % r2_score(y, pred))

['date_delta' 'temp' 'humidity' 'windspeed' 'workingday' 'holiday'
 'weekend' 'clear_few_clouds' 'cloudy_mist'
 'heacy_rain_thunderstorm_snow_icepallets' 'ligth_snow_rain_thunderstorm']
Coefficients: 
 [ 156.1116531   425.19345077 -127.98886775  -50.16651336 -208.51152204
   36.86079261  200.62864995    2.1976262    14.04291676    1.65648577
  -33.35955477]
Intercept: 
 236.40779304780767
On Training set:
Mean squared error: 205017.84
Variance of target variable: 616464.24
Variance explained: 0.67


In [16]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

In [77]:
from sklearn.preprocessing import StandardScaler
#Standardisierung

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [78]:
reg_ridg = linear_model.Ridge(alpha=9)
reg_ridg.fit(X_train, y_train)
y_train_pred = reg_ridg.predict(X_train)
y_test_pred = reg_ridg.predict(X_test)
# Evaluation


In [79]:
r2_test = r2_score(y_test, y_test_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

In [80]:
reg_ridg.coef_

array([1597.78378192,  603.49023281,  541.98222534,  -62.98182829,
        -72.9731173 ,    3.78062338,  -26.9592261 ,    6.32957192,
         66.55011178,    7.31916262,  -30.43455012, -155.49416314])

In [84]:
#Training

model_lr = linear_model.LinearRegression()
model_lr.fit(X_train, y_train)
y_train_pred = model_lr.predict(X_train)
y_test_pred = model_lr.predict(X_test)

In [89]:
model_lr.coef_

array([1618.31747445,  609.80671726,  549.79978824,  -68.67440765,
        -72.89761036,    3.43469728,  -26.97906282,    6.6920001 ,
         64.39610328,    9.52497215,  -30.20702079, -154.72335186])

In [20]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [85]:
r2_test = r2_score(y_test, y_test_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

In [86]:
print(r2_test)

0.731208800795897


In [87]:
print(mse_test)

1309393.9976462615


In [88]:
print(mae_test)

836.8425056112676


problem cannot turn datetime into float